In [154]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import os
from dotenv import load_dotenv

##### Twitter認証

In [155]:
# Twitter情報

load_dotenv()

# API情報
CK = os.environ['CONSUMER_KEY']
CS = os.environ['CONSUMER_SECRET']
AT = os.environ['ACCESS_TOKEN']
ATS = os.environ['ACCESS_TOKEN_SECRET']

#Twitterの認証
auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, ATS)
api = tweepy.API(auth)

#　”wait_on_rate_limit = True”　利用制限にひっかかた時に必要時間待機する
api=tweepy.API(auth,wait_on_rate_limit=True)

##### ↓次のセルを変更
- Account...@から始まるアカウント名を指定
- req_tweet_cnt...取得するツイート数を指定
  - 1500 requests / 15 mins PER APP
  - 900 requests / 15 mins PER USER

In [156]:
#取得するツイートの数と、検索するアカウントの入力
Account= '@sakuramiko35'
req_tweet_cnt = 150

##### 必要なTweetデータを取得
api.user_timeline： 特定のユーザーのタイムライン情報を取得

``` python
tweepy.Cursor(api.user_timeline, id="アカウント名").items(取得上限)
```

In [157]:
#検索条件を元にツイートを抽出
tweets = tweepy.Cursor(api.user_timeline, id=Account).items(req_tweet_cnt)

#### 取得できる時間データは世界時間なので日本時間に形式を変換

In [ ]:
#世界時間を日本時間に変換する
def change_time_JST(time):
    utc = datetime(time.year, time.month,time.day, \
    time.hour,time.minute,time.second, tzinfo=timezone.utc)
    jst = utc.astimezone(pytz.timezone("Asia/Tokyo"))
    str_time = jst.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time

##### Tweetデータを格納

In [158]:
# 取得したデータを格納する配列
tweet_data = []

# データを格納するイテレーション
for tweet in tweets:
    #ツイート時刻とユーザのアカウント作成時刻を日本時刻にする
    tweet_time = change_time_JST(tweet.created_at)
    # create_account_time = change_time_JST(tweet.user.created_at)

    # リツイートとリプライを除外しながらデータを取得
    if not tweet.favorite_count == 0 and not "@" in tweet.text:
        #tweet_dataの配列に取得したい情報を入れていく
        tweet_data.append([
            tweet.id,
            tweet_time,
            tweet.text,
            tweet.favorite_count,
            tweet.retweet_count,
            # tweet.user.screen_name,
            # tweet.user.name,
            # tweet.user.description,
            # tweet.user.friends_count,
            # tweet.user.followers_count,
            # create_account_time,
            # tweet.in_reply_to_status_id_str,
            # tweet.in_reply_to_user_id_str,
            # tweet.coordinates,
            # tweet.user.verified
        ])

Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id


##### 取り出したデータをpandasのDataFrameに変換

In [159]:
#CSVファイルに出力するときの列の名前を定義
columns=[
    'ツイートID',
    'ツイートされた時間(日本時間)',
    'ツイートの内容',
    'いいねの数',
    'リツイートの数',
    # 'ツイートした人のID',
    # 'アカウント名',
    # '自己紹介文',
    # 'フォロー数',
    # 'フォロワー数',
    # 'アカウント作成日時',
    # 'ツイートが返信の場合、返信先のツイートID',
    # 'ツイートが返信の場合、返信先のユーザーID',
    # 'ツイートされた場所',
    # '認証アカウント(公式アカウント)かどうか'
    ]

#tweet_dataのリストをpandasのDataFrameに変換
df = pd.DataFrame(tweet_data, columns=columns)

##### CSVファイルに出力する

In [ ]:
#CSVファイルの名前を決める
file_name='tweet.csv'

#CSVファイルを出力する
df.to_csv(file_name,encoding='utf-8-sig',index=False)